## Analyzing 5 Different Host Strucutres and Migration Graphs 
* $\ce{MnO2}$
* $\ce{VPO5}$ 
* $\ce{NaV4O10}$
* $\ce{Cu5P3O13}$
* $\ce{CrO3}$

By Tasha Lewis
January 1st, 2023

In [5]:
from pymatgen.core import Structure, Element, Lattice, PeriodicSite
from pymatgen.ext.matproj import MPRester
mpr = MPRester()
from pymatgen.analysis.diffusion.neb.full_path_mapper import MigrationGraph
from pprint import pprint
import numpy as np
from pydash import get

/Users/tashalewis/miniconda3/envs/torl/lib/python3.9/site-packages/pymatgen/analysis/path_finder.py:37: FutureWarning: This code has been superseded by pymatgen.analysis.neb in the separate add-on packagepymatgen-diffusion. This module here is retained for backwards compatibility. It will be removed from2022.1.1.
  warnings.warn(


In [6]:
from maggma.stores.advanced_stores import MongograntStore
from maggma.stores.compound_stores import ConcatStore

In [7]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import VoronoiNN, CrystalNN
vnn = VoronoiNN()
cnn = CrystalNN()
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()

In [8]:
# material_id and task_ids
materials = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "materials_2022", key="task_id")
materials.connect()

# "material_ids" & "host_material_ids" & "insertion_material_ids" 
sgroup = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "sgroups_2022", key="_id")
sgroup.connect()

# "material_ids" & "host_structure" & "battery_id" & "entries_composition_summary" 
insert_elec = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "insertion_electrodes_2022", key="_id")
insert_elec.connect()

# "battery_id" & "migration_graph" & "matrix_supercell_structure" & "inserted_ion_coords
mig_graph = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "migration_graph_2022", key="task_id")
mig_graph.connect()

aneba_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "approx_neb_analysis", key="wf_uuid")
aneba_store.connect()

elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "vw_elec", key="battid")
elec_store.connect()

# "battery_id" & "host_structure" & "material_ids" & "migration_graph" & "inserted_ion_coords"
new_elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "rank_electrodes_2022")
new_elec_store.connect()


In [61]:
def information(battery_id):
    old_batt_id = battery_id
    doc = aneba_store.query_one({"battid": battery_id})
    wf_uuid = doc["wf_uuid"]
    struct = Structure.from_dict(doc['host']['input'])
    
    elements = list(struct.composition.as_dict().keys())
    comp = struct.composition.to_pretty_string()
    
    elec_doc = elec_store.query_one({"approx_neb_wf_uuid":wf_uuid, 'battid': old_batt_id})
    
    if elec_doc:
        material_ids = [str(i) for i in elec_doc['material_ids']]   # get material ids
        js_material_ids = ['js-' + i for i in material_ids]
    
    new_elec_doc = new_elec_store.query_one({'material_ids':{'$in':js_material_ids},'elements':{'$all':elements}})
    host_struct = Structure.from_dict(new_elec_doc['host_structure'])
    #mg = MigrationGraph.from_dict(elec_doc['migration_graph'])
            
            
    return {"old_batt_id":battery_id,
            "wf_uuid":wf_uuid,
            "elements": elements,
            #"comp":comp,
            "js_material_ids": js_material_ids,
            #"doc":new_elec_doc ,
            "host_struct": host_struct
            
           }

In [62]:
information('113091_Mg')

{'old_batt_id': '113091_Mg',
 'wf_uuid': '938826bf-676d-473e-b98f-0c3c9f842eed',
 'elements': ['Mn', 'O'],
 'js_material_ids': ['js-72614',
  'js-113091',
  'js-113096',
  'js-47836',
  'js-47314',
  'js-46959',
  'js-46581',
  'js-18'],
 'host_struct': Structure Summary
 Lattice
     abc : 5.832981085441817 5.8254111932647294 5.837074515975361
  angles : 60.01961709382035 59.89097940572301 60.089408967637354
  volume : 140.24737212088203
       A : 5.050436 -0.001915 2.918349
       B : 1.670177 4.756793 2.918706
       C : 0.008962 -0.012617 5.837054
     pbc : True True True
 PeriodicSite: Mn (3.3648, 2.3711, 5.8371) [0.5000, 0.5000, 0.5000]
 PeriodicSite: Mn (0.8396, 2.3721, 4.3779) [0.0000, 0.5000, 0.5000]
 PeriodicSite: Mn (3.3603, 2.3774, 2.9185) [0.5000, 0.5000, 0.0000]
 PeriodicSite: Mn (2.5297, -0.0073, 4.3777) [0.5000, 0.0000, 0.5000]
 PeriodicSite: O (2.7230, 3.4828, 7.2998) [0.2950, 0.7343, 0.7359]
 PeriodicSite: O (4.9466, 3.4900, 8.5782) [0.7348, 0.7359, 0.7343]
 Periodi

## $\ce{MnO2}$

## $\ce{VPO5}$ 

## $\ce{NaV4O10}$

## $\ce{Cu5P3O13}$

## $\ce{CrO3}$